# Dataset 3: Delhi Hospital EEG Data Exploration

This notebook provides an initial exploration of the Delhi Hospital EEG dataset (Dataset 3).

## Dataset Overview

The Delhi Hospital dataset contains EEG recordings in `.mat` format, organized into three categories:
- **Pre-ictal**: Recordings before seizure onset
- **Interictal**: Recordings between seizures
- **Ictal**: Recordings during seizures

## Prerequisites

Before running this notebook:
1. Ensure Dataset 3 files are downloaded and placed in `data/raw/delhi_hospital_mat/`
2. Install required dependencies: `pip install -r requirements.txt`

**Note**: This notebook includes safeguards to handle missing data gracefully, so it can be run even before data is downloaded.

In [ ]:
import sys
from pathlib import Path
import warnings

# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

warnings.filterwarnings('ignore')

print("✓ Imports successful")

## Import Data Loader Utilities

In [ ]:
from loaders import (
    load_delhi_segment,
    load_multiple_segments,
    list_available_files,
    get_segment_info,
)
from loaders.dataset3_loader import extract_data_array, get_data_path

print("✓ Loader utilities imported successfully")

## Check Data Availability

First, let's check if the data files are present and list what's available.

In [ ]:
data_path = get_data_path()
print(f"Expected data path: {data_path}")
print(f"Data directory exists: {data_path.exists()}")
print()

# List available files
available_files = list_available_files()

print("Available files by category:")
print(f"  Pre-ictal: {len(available_files['pre_ictal'])} files")
print(f"  Interictal: {len(available_files['interictal'])} files")
print(f"  Ictal: {len(available_files['ictal'])} files")
print()

total_files = sum(len(files) for files in available_files.values())

if total_files == 0:
    print("⚠️  WARNING: No data files found!")
    print("")
    print("To continue with this exploration:")
    print("1. Download Dataset 3 (Delhi Hospital) files")
    print(f"2. Place .mat files in: {data_path}")
    print("3. Re-run this notebook")
    print("")
    print("The notebook will now demonstrate the analysis workflow with mock data.")
    DATA_AVAILABLE = False
else:
    print("✓ Data files found! Proceeding with exploration...")
    DATA_AVAILABLE = True

## Load Sample Segments

Load a few sample segments from each category (pre-ictal, interictal, ictal) to explore the data structure.

In [ ]:
if DATA_AVAILABLE:
    # Load up to 2 samples from each category
    max_samples = 2
    
    print("Loading segments...")
    pre_ictal_segments = load_multiple_segments(available_files['pre_ictal'][:max_samples])
    interictal_segments = load_multiple_segments(available_files['interictal'][:max_samples])
    ictal_segments = load_multiple_segments(available_files['ictal'][:max_samples])
    
    print(f"Loaded {len(pre_ictal_segments)} pre-ictal segments")
    print(f"Loaded {len(interictal_segments)} interictal segments")
    print(f"Loaded {len(ictal_segments)} ictal segments")
    
    # Display info about the first segment from each category
    if pre_ictal_segments:
        print("\nPre-ictal segment info:")
        print(get_segment_info(pre_ictal_segments[0]))
    
    if interictal_segments:
        print("\nInterictal segment info:")
        print(get_segment_info(interictal_segments[0]))
    
    if ictal_segments:
        print("\nIctal segment info:")
        print(get_segment_info(ictal_segments[0]))
else:
    print("No data available. Creating mock data for demonstration purposes...")
    
    # Create mock data with realistic properties
    np.random.seed(42)
    n_channels = 23
    n_samples = 4097
    
    # Simulate different characteristics for each segment type
    pre_ictal_mock = {'data': np.random.randn(n_channels, n_samples) * 50}
    interictal_mock = {'data': np.random.randn(n_channels, n_samples) * 30}
    ictal_mock = {'data': np.random.randn(n_channels, n_samples) * 100 + np.sin(np.linspace(0, 20*np.pi, n_samples)) * 80}
    
    pre_ictal_segments = [pre_ictal_mock]
    interictal_segments = [interictal_mock]
    ictal_segments = [ictal_mock]
    
    print("✓ Mock data created")
    print(f"  Shape: ({n_channels} channels, {n_samples} samples)")

## Visualization: Channel-wise Line Plots

Display time-series plots for a subset of channels from each segment type.

In [ ]:
def plot_channels(segment_data, segment_type, n_channels_to_plot=5, max_samples=2000):
    """
    Plot multiple channels from a segment.
    
    Args:
        segment_data: Dictionary containing 'data' key with shape (n_channels, n_samples)
        segment_type: String describing the segment type (for title)
        n_channels_to_plot: Number of channels to display
        max_samples: Maximum number of samples to plot (for performance)
    """
    data = extract_data_array(segment_data)
    if data is None:
        print(f"Could not extract data array from {segment_type} segment")
        return
    
    n_channels = min(n_channels_to_plot, data.shape[0])
    n_samples_plot = min(max_samples, data.shape[1])
    
    fig, axes = plt.subplots(n_channels, 1, figsize=(14, 2*n_channels), sharex=True)
    if n_channels == 1:
        axes = [axes]
    
    time_axis = np.arange(n_samples_plot)
    
    for i, ax in enumerate(axes):
        ax.plot(time_axis, data[i, :n_samples_plot], linewidth=0.5)
        ax.set_ylabel(f'Ch {i+1}\n(μV)', fontsize=9)
        ax.grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Sample Index', fontsize=10)
    axes[0].set_title(f'{segment_type} Segment - Channel-wise EEG Signals', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()


# Plot one segment from each category
if pre_ictal_segments:
    plot_channels(pre_ictal_segments[0], 'Pre-ictal')

if interictal_segments:
    plot_channels(interictal_segments[0], 'Interictal')

if ictal_segments:
    plot_channels(ictal_segments[0], 'Ictal')

## Basic Statistics: Mean and Variance

Compute and compare basic statistical measures across segment types.

In [ ]:
def compute_basic_stats(segments, segment_type):
    """
    Compute mean and variance statistics for segments.
    
    Args:
        segments: List of segment dictionaries
        segment_type: String describing the segment type
    
    Returns:
        DataFrame with statistics
    """
    stats = []
    
    for idx, segment in enumerate(segments):
        data = extract_data_array(segment)
        if data is None:
            continue
        
        stats.append({
            'segment_type': segment_type,
            'segment_idx': idx,
            'mean': np.mean(data),
            'std': np.std(data),
            'variance': np.var(data),
            'min': np.min(data),
            'max': np.max(data),
            'median': np.median(data),
            'mean_abs': np.mean(np.abs(data)),
        })
    
    return pd.DataFrame(stats)


# Compute statistics for all segments
all_stats = []

if pre_ictal_segments:
    all_stats.append(compute_basic_stats(pre_ictal_segments, 'Pre-ictal'))

if interictal_segments:
    all_stats.append(compute_basic_stats(interictal_segments, 'Interictal'))

if ictal_segments:
    all_stats.append(compute_basic_stats(ictal_segments, 'Ictal'))

if all_stats:
    stats_df = pd.concat(all_stats, ignore_index=True)
    print("Basic Statistics Summary:")
    print(stats_df.to_string(index=False))
    print()
    
    # Group by segment type and show averages
    print("\nAveraged by Segment Type:")
    grouped = stats_df.groupby('segment_type').mean(numeric_only=True)
    print(grouped.to_string())

## Visualize Statistics Comparison

In [ ]:
if all_stats and len(stats_df) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Mean comparison
    stats_df.boxplot(column='mean', by='segment_type', ax=axes[0])
    axes[0].set_title('Mean Signal Amplitude')
    axes[0].set_xlabel('Segment Type')
    axes[0].set_ylabel('Mean (μV)')
    
    # Variance comparison
    stats_df.boxplot(column='variance', by='segment_type', ax=axes[1])
    axes[1].set_title('Signal Variance')
    axes[1].set_xlabel('Segment Type')
    axes[1].set_ylabel('Variance')
    
    # Standard deviation comparison
    stats_df.boxplot(column='std', by='segment_type', ax=axes[2])
    axes[2].set_title('Signal Standard Deviation')
    axes[2].set_xlabel('Segment Type')
    axes[2].set_ylabel('Std Dev (μV)')
    
    plt.suptitle('Statistical Comparison Across Segment Types', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

## Spectral Analysis: Band Power Summaries

Compute power spectral density (PSD) and extract power in different frequency bands:
- **Delta**: 0.5-4 Hz
- **Theta**: 4-8 Hz
- **Alpha**: 8-13 Hz
- **Beta**: 13-30 Hz
- **Gamma**: 30-50 Hz

In [ ]:
def compute_band_power(data, fs=256, bands=None):
    """
    Compute power in different frequency bands.
    
    Args:
        data: EEG data array with shape (n_channels, n_samples)
        fs: Sampling frequency in Hz
        bands: Dictionary of frequency bands {name: (low_freq, high_freq)}
    
    Returns:
        Dictionary with band power values
    """
    if bands is None:
        bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30),
            'gamma': (30, 50),
        }
    
    # Compute PSD using Welch's method
    freqs, psd = signal.welch(data, fs=fs, nperseg=min(256, data.shape[1]//4))
    
    # Average PSD across channels
    psd_mean = np.mean(psd, axis=0)
    
    # Compute band powers
    band_powers = {}
    for band_name, (low_freq, high_freq) in bands.items():
        idx_band = np.logical_and(freqs >= low_freq, freqs <= high_freq)
        band_powers[band_name] = np.trapz(psd_mean[idx_band], freqs[idx_band])
    
    return band_powers, freqs, psd_mean


def compute_spectral_stats(segments, segment_type, fs=256):
    """
    Compute spectral band power statistics for segments.
    
    Args:
        segments: List of segment dictionaries
        segment_type: String describing the segment type
        fs: Sampling frequency in Hz
    
    Returns:
        DataFrame with spectral statistics
    """
    spectral_stats = []
    
    for idx, segment in enumerate(segments):
        data = extract_data_array(segment)
        if data is None:
            continue
        
        band_powers, _, _ = compute_band_power(data, fs=fs)
        
        stats = {
            'segment_type': segment_type,
            'segment_idx': idx,
        }
        stats.update(band_powers)
        spectral_stats.append(stats)
    
    return pd.DataFrame(spectral_stats)


# Compute spectral statistics
all_spectral_stats = []

if pre_ictal_segments:
    all_spectral_stats.append(compute_spectral_stats(pre_ictal_segments, 'Pre-ictal'))

if interictal_segments:
    all_spectral_stats.append(compute_spectral_stats(interictal_segments, 'Interictal'))

if ictal_segments:
    all_spectral_stats.append(compute_spectral_stats(ictal_segments, 'Ictal'))

if all_spectral_stats:
    spectral_df = pd.concat(all_spectral_stats, ignore_index=True)
    print("Spectral Band Power Summary:")
    print(spectral_df.to_string(index=False))
    print()
    
    # Group by segment type
    print("\nAveraged by Segment Type:")
    grouped_spectral = spectral_df.groupby('segment_type').mean(numeric_only=True)
    print(grouped_spectral.to_string())

## Visualize Spectral Band Powers

In [ ]:
if all_spectral_stats and len(spectral_df) > 0:
    # Prepare data for visualization
    band_columns = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    
    # Create a bar plot comparing band powers
    fig, ax = plt.subplots(figsize=(12, 6))
    
    grouped_spectral = spectral_df.groupby('segment_type')[band_columns].mean()
    grouped_spectral.plot(kind='bar', ax=ax, width=0.8)
    
    ax.set_title('Average Spectral Band Power by Segment Type', fontsize=14, fontweight='bold')
    ax.set_xlabel('Segment Type', fontsize=12)
    ax.set_ylabel('Power (μV²/Hz)', fontsize=12)
    ax.legend(title='Frequency Band', fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=0)
    
    plt.tight_layout()
    plt.show()

## Power Spectral Density Plots

In [ ]:
def plot_psd_comparison(segments_dict, fs=256):
    """
    Plot PSD comparison across different segment types.
    
    Args:
        segments_dict: Dictionary mapping segment type names to lists of segments
        fs: Sampling frequency
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    for segment_type, segments in segments_dict.items():
        if not segments:
            continue
        
        # Use first segment
        data = extract_data_array(segments[0])
        if data is None:
            continue
        
        _, freqs, psd_mean = compute_band_power(data, fs=fs)
        
        # Plot in dB scale
        ax.semilogy(freqs, psd_mean, label=segment_type, linewidth=2, alpha=0.8)
    
    ax.set_xlabel('Frequency (Hz)', fontsize=12)
    ax.set_ylabel('Power Spectral Density (μV²/Hz)', fontsize=12)
    ax.set_title('Power Spectral Density Comparison', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 50])  # Focus on 0-50 Hz range
    
    plt.tight_layout()
    plt.show()


# Plot PSD comparison
segments_for_psd = {}
if pre_ictal_segments:
    segments_for_psd['Pre-ictal'] = pre_ictal_segments
if interictal_segments:
    segments_for_psd['Interictal'] = interictal_segments
if ictal_segments:
    segments_for_psd['Ictal'] = ictal_segments

if segments_for_psd:
    plot_psd_comparison(segments_for_psd)

## Summary and Key Observations

This exploratory analysis provides an initial look at the Delhi Hospital EEG dataset. Key points to observe:

1. **Signal Characteristics**: Compare amplitude ranges and variability across segment types
2. **Statistical Differences**: Note differences in mean, variance, and distribution between pre-ictal, interictal, and ictal segments
3. **Spectral Patterns**: Observe which frequency bands show the most power in different segment types
4. **Seizure Signatures**: Ictal segments typically show distinct patterns in both time and frequency domains

## Next Steps: Extending the Exploration

To further explore this dataset, consider:

### 1. Advanced Feature Extraction
- **Time-domain features**: Line length, Hjorth parameters, zero-crossing rate
- **Frequency-domain features**: Spectral entropy, spectral edge frequency, relative band powers
- **Time-frequency features**: Wavelet coefficients, short-time Fourier transform (STFT)
- **Non-linear features**: Approximate entropy, sample entropy, Lyapunov exponents

### 2. Data Preprocessing
- Apply band-pass filtering (e.g., 0.5-50 Hz) to remove artifacts
- Implement artifact rejection algorithms (e.g., for eye blinks, muscle artifacts)
- Normalize or standardize signals
- Apply Independent Component Analysis (ICA) for artifact removal

### 3. Visualization Enhancements
- Create spectrograms to visualize time-frequency patterns
- Plot topographic maps if electrode positions are available
- Add correlation matrices between channels
- Create interactive plots with Plotly for better exploration

### 4. Statistical Analysis
- Perform hypothesis testing to confirm differences between segment types
- Calculate effect sizes for key features
- Analyze channel-specific patterns
- Investigate temporal patterns within segments

### 5. Machine Learning Preparation
- Create a balanced dataset with equal samples from each category
- Split data into training, validation, and test sets
- Design a feature extraction pipeline
- Implement cross-validation strategies

### 6. Segment Analysis
- Analyze the temporal progression in pre-ictal segments
- Study the onset and offset patterns in ictal segments
- Compare patient-specific patterns
- Investigate segment duration statistics

### Example Code Template for Extensions

```python
# Example: Add band-pass filtering
from scipy.signal import butter, filtfilt

def bandpass_filter(data, lowcut=0.5, highcut=50, fs=256, order=4):
    nyquist = fs / 2
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_data = filtfilt(b, a, data, axis=1)
    return filtered_data

# Example: Extract Hjorth parameters
def hjorth_parameters(signal):
    first_deriv = np.diff(signal)
    second_deriv = np.diff(first_deriv)
    
    var_signal = np.var(signal)
    var_first_deriv = np.var(first_deriv)
    var_second_deriv = np.var(second_deriv)
    
    activity = var_signal
    mobility = np.sqrt(var_first_deriv / var_signal)
    complexity = np.sqrt(var_second_deriv / var_first_deriv) / mobility
    
    return activity, mobility, complexity
```

### Resources

For more information on EEG analysis and seizure detection:
- MNE-Python documentation: https://mne.tools/
- EEGLAB: https://sccn.ucsd.edu/eeglab/
- NeuroKit2: https://neuropsychology.github.io/NeuroKit/

---

**Happy exploring! 🧠⚡**